In [3]:
!pip install tensorflow==2.3.1
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
# TODO: Change Models You want https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet
from tensorflow.keras.applications.resnet import ResNet152
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import numpy as np

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.1


## **CIFAR 10 Datast 데이터셋 다운로드하고 준비하기**




In [4]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  # TODO: Change Models You want to change
  output_ims = tf.keras.applications.resnet.preprocess_input(input_images)
  return output_ims

In [5]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [6]:
train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [7]:
train_X.shape

(50000, 32, 32, 3)

In [8]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  #Change Models Here too!!!
  
  feature_extractor = tf.keras.applications.resnet.ResNet152(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='Adam', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

234700800/234698864 [==============================] - 3s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
resnet152 (Functional)       (None, 7, 7, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_____________________________________________________

In [9]:
EPOCHS = 3
history = model.fit(train_X, training_labels, epochs=EPOCHS, validation_data = (valid_X, validation_labels), batch_size=64)

Epoch 1/3
782/782 [==============================] - 1344s 2s/step - loss: 1.1061 - accuracy: 0.6054 - val_loss: 1.1018 - val_accuracy: 0.6417
Epoch 2/3
782/782 [==============================] - 1360s 2s/step - loss: 0.6380 - accuracy: 0.7792 - val_loss: 0.7118 - val_accuracy: 0.7574
Epoch 3/3
782/782 [==============================] - 1355s 2s/step - loss: 0.4468 - accuracy: 0.8448 - val_loss: 0.4937 - val_accuracy: 0.8304


In [10]:
loss, accuracy = model.evaluate(valid_X, validation_labels, batch_size=64)

157/157 [==============================] - 80s 508ms/step - loss: 0.4937 - accuracy: 0.8304
